# v4

In [83]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import GradScaler, autocast
from torch.optim import Adam



def random_projection(matrix, k):
    """Random projection to reduce dimensionality of matrix to k dimensions."""
    random_matrix = torch.randn(matrix.size(-1), k, device=matrix.device)
    return torch.matmul(matrix, random_matrix)

def cur_decomposition(matrix, k):
    """Applies CUR decomposition on each head for each batch."""
    batch_size, seq_length, heads, dim = matrix.shape
    # Ensuring k does not exceed dimensions
    k = min(k, dim)
    C = torch.zeros(batch_size, seq_length, heads, k, device=matrix.device)
    R = torch.zeros(batch_size, k, heads, dim, device=matrix.device)
    
    for b in range(batch_size):
        for h in range(heads):
            col_indices = np.random.choice(dim, k, replace=False)
            row_indices = np.random.choice(seq_length, k, replace=False)
            C[b, :, h] = matrix[b, :, h, col_indices]
            R[b, :, h] = matrix[b, row_indices, h]
    return C, R

class PartitionedLinformerAttentionACT(nn.Module):
    def __init__(self, embed_size, heads, sequence_length, projection_dim, partition_size):
        super().__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.sequence_length = sequence_length
        self.projection_dim = projection_dim
        self.partition_size = partition_size
        self.head_dim = embed_size // heads

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.projection_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.projection_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)
        self.ponder = nn.Linear(self.partition_size, 1, bias=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        N, seq_length, _ = x.shape
        print(f"x shape as input to forward: {x.shape}")
        print(f"embed size as input to forward: {self.embed_size}")

        x_reshaped = x.view(N, seq_length, self.heads, self.head_dim)
        print(f"x_reshaped = x.view(N, seq_length, self.heads, self.head_dim): {x_reshaped.shape}")

        values = self.values(x_reshaped).view(N, seq_length, self.heads, self.head_dim)
        keys = self.keys(x_reshaped).view(N, seq_length, self.heads, self.projection_dim)
        queries = self.queries(x_reshaped).view(N, seq_length, self.heads, self.projection_dim)
        print(f"values: {values.shape}, queries: {queries.shape}, keys: {keys.shape}")

        keys = random_projection(keys, self.projection_dim // 2)
        queries = random_projection(queries, self.projection_dim // 2)
        print(f"queries after random projection: {queries.shape}, keys after random projection: {keys.shape}")
        attention_scores = torch.zeros(N, self.heads, seq_length, self.projection_dim // 2, device=x.device)
        print(f"attention_scores after random projection: {attention_scores.shape}")

        for i in range(0, seq_length, self.partition_size):
            print(f"PARTITION START")
            partition_start = i
            partition_end = min(i + self.partition_size, seq_length)
            keys_partition = keys[:, partition_start:partition_end, :, :]
            queries_partition = queries[:, partition_start:partition_end, :, :]

            C_keys, R_queries = cur_decomposition(keys_partition, k=keys_partition.size(-1) // 2)

            ponder_scores = torch.zeros(N, self.heads, partition_end - partition_start, 1, device=x.device)
            print(f"Partition Start {i}, Partition End {partition_end} ,keys_partition: {keys_partition.shape}, queries_partition: {queries_partition.shape}, ponder_scores: {ponder_scores.shape}")

            for h in range(self.heads):
                print(f"HEADS START")
                head_queries = queries_partition[:, :, h, :]
                print(f"head_queries: {head_queries.shape}")

                # Apply the ponder network
                head_ponder_scores = self.sigmoid(self.ponder(head_queries))
                print(f"head_ponder_scores shape before view: {head_ponder_scores.shape}")
                # Reshape head_ponder_scores back to match the original batch and partition sizes
                # and expand the last dimension to match the expected shape for broadcasting
                head_ponder_scores_reshaped = head_ponder_scores.view(N, partition_end - partition_start, -1).expand(-1, -1, self.head_dim)
                print(f"head_ponder_scores_reshaped shape : {head_ponder_scores_reshaped.shape}")

                # Assign the reshaped ponder scores back to the corresponding head in the ponder_scores tensor
                ponder_scores[:, h, :, 0] = head_ponder_scores_reshaped 

            energy = torch.einsum('bnhdp,bnhpd->bnhdp', queries_partition, C_keys.transpose(-2, -1))
            attention = F.softmax(energy, dim=-1) * ponder_scores
            attention_scores[:, :, partition_start:partition_end, :] = attention

        out = torch.einsum('bnhdp,bnhpd->bnshp', attention_scores, values).reshape(N, seq_length, -1)
        return self.fc_out(out)




# Example usage

model = PartitionedLinformerAttentionACT(embed_size=512, heads=8, sequence_length=1024, projection_dim=256, partition_size=128)
input_tensor = torch.rand(2, model.sequence_length, model.embed_size)
output = model(input_tensor)
print(output.shape)  # Expected output shape: [2, sequence_length, embed_size]


# Create a DataLoader with pinned memory
class MyDataset(Dataset):

    def __init__(self):

        # Initialize your data here

        pass

    

    def __len__(self):

        # Return the size of your dataset

        return 100  # Example size



    def __getitem__(self, idx):

        # Implement logic to get a single item at a given index

        # For simplicity, let's return random tensors

        return torch.randn(10, 64), torch.randint(0, 2, (10,))  # Example data

dataset = MyDataset()
loader = DataLoader(dataset, batch_size=32, shuffle=True, pin_memory=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = Adam(model.parameters(), lr=1e-3)
scaler = GradScaler()

for epoch in range(10):  # Example: 10 epochs
    for inputs, targets in loader:
        inputs, targets = inputs.to(device, non_blocking=True), targets.to(device, non_blocking=True)
        optimizer.zero_grad()
        with autocast():
            predictions = model(inputs)
            loss = nn.functional.cross_entropy(predictions, targets)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()



x shape as input to forward: torch.Size([2, 1024, 512])
embed size as input to forward: 512
x_reshaped = x.view(N, seq_length, self.heads, self.head_dim): torch.Size([2, 1024, 8, 64])
values: torch.Size([2, 1024, 8, 64]), queries: torch.Size([2, 1024, 8, 256]), keys: torch.Size([2, 1024, 8, 256])
queries after random projection: torch.Size([2, 1024, 8, 128]), keys after random projection: torch.Size([2, 1024, 8, 128])
attention_scores after random projection: torch.Size([2, 8, 1024, 128])
PARTITION START
Partition Start 0, Partition End 128 ,keys_partition: torch.Size([2, 128, 8, 128]), queries_partition: torch.Size([2, 128, 8, 128]), ponder_scores: torch.Size([2, 8, 128, 1])
HEADS START
head_queries: torch.Size([2, 128, 128])
head_ponder_scores shape before view: torch.Size([2, 128, 1])
head_ponder_scores_reshaped shape : torch.Size([2, 128, 64])


RuntimeError: expand(torch.FloatTensor{[2, 128, 64]}, size=[2, 128]): the number of sizes provided (2) must be greater or equal to the number of dimensions in the tensor (3)

#v2

In [107]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import GradScaler, autocast
from torch.optim import Adam



def random_projection(matrix, k):
    """Random projection to reduce dimensionality of matrix to k dimensions."""
    random_matrix = torch.randn(matrix.size(-1), k, device=matrix.device)
    return torch.matmul(matrix, random_matrix)

def cur_decomposition(matrix, projection_dim):  # Change 'k' argument
    """Applies CUR decomposition with C matrix dimension aligned to projection dimension."""
    batch_size, seq_length, heads, dim = matrix.shape
    k = min(projection_dim // 2, dim)  # Use projection dimension to determine 'k'
    C = torch.zeros(batch_size, seq_length, heads, k, device=matrix.device)
    R = torch.zeros(batch_size, k, heads, dim, device=matrix.device)

    for b in range(batch_size):
        for h in range(heads):
            col_indices = np.random.choice(dim, k, replace=False)
            row_indices = np.random.choice(seq_length, k, replace=False)
            C[b, :, h] = matrix[b, :, h, col_indices]
            R[b, :, h] = matrix[b, row_indices, h]
    return C, R

class PartitionedLinformerAttentionACT(nn.Module):
    def __init__(self, embed_size, heads, sequence_length, projection_dim, partition_size):
        super().__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.sequence_length = sequence_length
        self.projection_dim = projection_dim
        self.partition_size = partition_size
        self.head_dim = embed_size // heads

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.projection_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.projection_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)
        self.ponder = nn.Linear(self.partition_size, 1, bias=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        N, seq_length, _ = x.shape
        print(f"x shape as input to forward: {x.shape}")
        print(f"embed size as input to forward: {self.embed_size}")

        x_reshaped = x.view(N, seq_length, self.heads, self.head_dim)
        print(f"x_reshaped = x.view(N, seq_length, self.heads, self.head_dim): {x_reshaped.shape}")

        values = self.values(x_reshaped)  # Don't need the extra view
        queries = random_projection(self.queries(x_reshaped), self.projection_dim // 2)
        keys = random_projection(self.keys(x_reshaped), self.projection_dim // 2)
        print(f"values: {values.shape}, queries: {queries.shape}, keys: {keys.shape}")

        attention_scores = torch.zeros(N, self.heads, seq_length, self.projection_dim // 2, device=x.device)
        print(f"attention_scores after random projection: {attention_scores.shape}")

        for i in range(0, seq_length, self.partition_size):
            print(f"PARTITION START")
            partition_start = i
            partition_end = min(i + self.partition_size, seq_length)
            
            # Extract partition slices
            keys_part = keys[:, partition_start:partition_end, :, :]
            queries_part = queries[:, partition_start:partition_end, :, :]

            C_keys, R_queries = cur_decomposition(keys_part, self.projection_dim)
            print("C_keys shape before return:", C_keys.shape)

            ponder_scores = torch.zeros(N, self.heads, partition_end - partition_start, 1, device=x.device)
            print(f"Partition Start {i}, Partition End {partition_end} , ponder_scores: {ponder_scores.shape}")

            for h in range(self.heads):
                print(f"HEADS START")
            
                head_queries = queries_part[:, :, h, :]
                print(f"head_queries: {head_queries.shape}")

                head_ponder_scores = self.sigmoid(self.ponder(head_queries))  # Shape: [N, partition_size, 1]
                print(f"head_ponder_scores: {head_ponder_scores.shape}")

                ponder_scores[:, h, :, 0] = head_ponder_scores[:, :, 0]  # Assign only the relevant scores 

            ponder_scores = ponder_scores.permute(0, 2, 1, 3)  # New shape: [batch, partition_size, heads, 1]
            print("BEFORE EINSUM:")
            print("queries_part shape:", queries_part.shape) 
            print("C_keys shape:", C_keys.shape)
            ponder_scores_broadcastable = ponder_scores.expand(-1, -1, -1, self.projection_dim // 2)

            energy = torch.einsum('bnhd,bnhk->bnhd', queries_part, C_keys)
            print("AFTER EINSUM:")
            print(f"energy shape: {energy.shape}")
            print(f"ponder_scores shape: {ponder_scores.shape}")
            # Apply softmax over the last dimension of energy
            attention_weights = F.softmax(energy, dim=-1)

            # Multiply by broadcasted ponder_scores
            attention = attention_weights * ponder_scores_broadcastable
            attention_scores[:, :, partition_start:partition_end, :] = attention.transpose(1, 2)
        print("attention_scores shape:", attention_scores.shape)
        print("values shape:", values.shape)

        out = torch.einsum('bnhp,bnhp->bnh', attention_scores, values)
        return self.fc_out(out)


# Example usage
model = PartitionedLinformerAttentionACT(embed_size=512, heads=8, sequence_length=1024, projection_dim=256, partition_size=128)
input_tensor = torch.rand(2, model.sequence_length, model.embed_size)
output = model(input_tensor)
print(output.shape)  # Expected output shape: [2, sequence_length, embed_size]


# Create a DataLoader with pinned memory
class MyDataset(Dataset):

    def __init__(self):

        # Initialize your data here

        pass

    

    def __len__(self):

        # Return the size of your dataset

        return 100  # Example size



    def __getitem__(self, idx):

        # Implement logic to get a single item at a given index

        # For simplicity, let's return random tensors

        return torch.randn(10, 64), torch.randint(0, 2, (10,))  # Example data

dataset = MyDataset()
loader = DataLoader(dataset, batch_size=32, shuffle=True, pin_memory=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = Adam(model.parameters(), lr=1e-3)
scaler = GradScaler()

for epoch in range(10):  # Example: 10 epochs
    for inputs, targets in loader:
        inputs, targets = inputs.to(device, non_blocking=True), targets.to(device, non_blocking=True)
        optimizer.zero_grad()
        with autocast():
            predictions = model(inputs)
            loss = nn.functional.cross_entropy(predictions, targets)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()



x shape as input to forward: torch.Size([2, 1024, 512])
embed size as input to forward: 512
x_reshaped = x.view(N, seq_length, self.heads, self.head_dim): torch.Size([2, 1024, 8, 64])
values: torch.Size([2, 1024, 8, 64]), queries: torch.Size([2, 1024, 8, 128]), keys: torch.Size([2, 1024, 8, 128])
attention_scores after random projection: torch.Size([2, 8, 1024, 128])
PARTITION START
C_keys shape before return: torch.Size([2, 128, 8, 128])
Partition Start 0, Partition End 128 , ponder_scores: torch.Size([2, 8, 128, 1])
HEADS START
head_queries: torch.Size([2, 128, 128])
head_ponder_scores: torch.Size([2, 128, 1])
HEADS START
head_queries: torch.Size([2, 128, 128])
head_ponder_scores: torch.Size([2, 128, 1])
HEADS START
head_queries: torch.Size([2, 128, 128])
head_ponder_scores: torch.Size([2, 128, 1])
HEADS START
head_queries: torch.Size([2, 128, 128])
head_ponder_scores: torch.Size([2, 128, 1])
HEADS START
head_queries: torch.Size([2, 128, 128])
head_ponder_scores: torch.Size([2, 128, 

RuntimeError: einsum(): subscript n has size 1024 for operand 1 which does not broadcast with previously seen size 8

# v3

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import GradScaler, autocast
from torch.optim import Adam



def random_projection(matrix, k):
    """Random projection to reduce dimensionality of matrix to k dimensions."""
    random_matrix = torch.randn(matrix.size(-1), k, device=matrix.device)
    return torch.matmul(matrix, random_matrix)

def cur_decomposition(matrix, projection_dim):  # Change 'k' argument
    """Applies CUR decomposition with C matrix dimension aligned to projection dimension."""
    batch_size, seq_length, heads, dim = matrix.shape
    k = min(projection_dim // 2, dim)  # Use projection dimension to determine 'k'
    C = torch.zeros(batch_size, seq_length, heads, k, device=matrix.device)
    R = torch.zeros(batch_size, k, heads, dim, device=matrix.device)

    for b in range(batch_size):
        for h in range(heads):
            col_indices = np.random.choice(dim, k, replace=False)
            row_indices = np.random.choice(seq_length, k, replace=False)
            C[b, :, h] = matrix[b, :, h, col_indices]
            R[b, :, h] = matrix[b, row_indices, h]
    return C, R

class PartitionedLinformerAttentionACT(nn.Module):
    def __init__(self, embed_size, heads, sequence_length, projection_dim, partition_size):
        super().__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.sequence_length = sequence_length
        self.projection_dim = projection_dim
        self.partition_size = partition_size
        self.head_dim = embed_size // heads

        # Linear transformations
        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.projection_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.projection_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)
        self.value_projection = nn.Linear(self.head_dim, self.projection_dim)  # New linear transformation to match dimensions
        self.ponder = nn.Linear(self.partition_size, 1, bias=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        N, seq_length, _ = x.shape
        x_reshaped = x.view(N, seq_length, self.heads, self.head_dim)

        values = self.values(x_reshaped)
        queries = random_projection(self.queries(x_reshaped), self.projection_dim // 2)
        keys = random_projection(self.keys(x_reshaped), self.projection_dim // 2)
        
        # Projecting values to match the projection dimension
        projected_values = self.value_projection(values)  # Transforming values here

        attention_scores = torch.zeros(N, self.heads, seq_length, self.projection_dim, device=x.device)
        print(f"attention_scores after random projection: {attention_scores.shape}")

        for i in range(0, seq_length, self.partition_size):
            print(f"PARTITION START")
            partition_start = i
            partition_end = min(i + self.partition_size, seq_length)
            
            # Extract partition slices
            keys_part = keys[:, partition_start:partition_end, :, :]
            queries_part = queries[:, partition_start:partition_end, :, :]

            C_keys, R_queries = cur_decomposition(keys_part, self.projection_dim)
            print("C_keys shape before return:", C_keys.shape)

            ponder_scores = torch.zeros(N, self.heads, partition_end - partition_start, 1, device=x.device)
            print(f"Partition Start {i}, Partition End {partition_end} , ponder_scores: {ponder_scores.shape}")

            for h in range(self.heads):
                print(f"HEADS START")
            
                head_queries = queries_part[:, :, h, :]
                print(f"head_queries: {head_queries.shape}")

                head_ponder_scores = self.sigmoid(self.ponder(head_queries))  # Shape: [N, partition_size, 1]
                print(f"head_ponder_scores: {head_ponder_scores.shape}")

                ponder_scores[:, h, :, 0] = head_ponder_scores[:, :, 0]  # Assign only the relevant scores 

            ponder_scores = ponder_scores.permute(0, 2, 1, 3)  # New shape: [batch, partition_size, heads, 1]
            print("BEFORE EINSUM:")
            print("queries_part shape:", queries_part.shape) 
            print("C_keys shape:", C_keys.shape)
            ponder_scores_broadcastable = ponder_scores.expand(-1, -1, -1, self.projection_dim // 2)

            energy = torch.einsum('bnhd,bnhk->bnhd', queries_part, C_keys)
            print("AFTER EINSUM:")
            print(f"energy shape: {energy.shape}")
            print(f"ponder_scores shape: {ponder_scores.shape}")
            # Apply softmax over the last dimension of energy
            attention_weights = F.softmax(energy, dim=-1)

            # Multiply by broadcasted ponder_scores
            attention = attention_weights * ponder_scores_broadcastable
            attention_scores[:, :, partition_start:partition_end, :] = attention.transpose(1, 2)
        print("attention_scores shape:", attention_scores.shape)
        print("values shape:", values.shape)

        out = torch.einsum('bnhp,bnhp->bnh', attention_scores, projected_values)
        return self.fc_out(out.view(N, seq_length, -1))


# Example usage
model = PartitionedLinformerAttentionACT(embed_size=512, heads=8, sequence_length=1024, projection_dim=256, partition_size=128)
input_tensor = torch.rand(2, model.sequence_length, model.embed_size)
output = model(input_tensor)
print(output.shape)  # Expected output shape: [2, sequence_length, embed_size]


# Create a DataLoader with pinned memory
class MyDataset(Dataset):

    def __init__(self):

        # Initialize your data here

        pass

    

    def __len__(self):

        # Return the size of your dataset

        return 100  # Example size



    def __getitem__(self, idx):

        # Implement logic to get a single item at a given index

        # For simplicity, let's return random tensors

        return torch.randn(10, 64), torch.randint(0, 2, (10,))  # Example data

dataset = MyDataset()
loader = DataLoader(dataset, batch_size=32, shuffle=True, pin_memory=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = Adam(model.parameters(), lr=1e-3)
scaler = GradScaler()

for epoch in range(10):  # Example: 10 epochs
    for inputs, targets in loader:
        inputs, targets = inputs.to(device, non_blocking=True), targets.to(device, non_blocking=True)
        optimizer.zero_grad()
        with autocast():
            predictions = model(inputs)
            loss = nn.functional.cross_entropy(predictions, targets)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

